In [1]:
# -*- coding: utf-8 -*-

from tika import parser
import shutil
import os, sys, subprocess
import re 

# hwp to txt 변환

pyhwp가 설치되어 있어야 한다. 하지만 스크립트 내에서 import하여 불러오진 않는다. 

## BOK - MPB minutes

In [200]:
hwp_path = r'.\data_files\BOK_minutes\hwp'
hwp_path = os.path.abspath(hwp_path)
hwp_path

'C:\\Users\\chlje\\VSCodeProjects\\asiae-BOK-nlp\\data_files\\BOK_minutes\\hwp'

In [201]:
hwp_tmp_path = r'.\data_files\BOK_minutes\hwp\tmp'
hwp_tmp_path = os.path.abspath(hwp_tmp_path)
hwp_tmp_path

'C:\\Users\\chlje\\VSCodeProjects\\asiae-BOK-nlp\\data_files\\BOK_minutes\\hwp\\tmp'

In [202]:
hwp2txt_path = r'.\data_files\BOK_minutes\hwp2txt'
hwp2txt_path = os.path.abspath(hwp2txt_path)
hwp2txt_path

'C:\\Users\\chlje\\VSCodeProjects\\asiae-BOK-nlp\\data_files\\BOK_minutes\\hwp2txt'

<font style='color:red;'>pyhwp를 쓰며 매우 이상했던 부분이다. </font>

- Terminal에서 실행할 경우:
    - hwp5txt 한글이포함된파일경로: 실행되지 않는다. 한글 부분이 ???로 들어가며 오류난다. 
    - hwp5txt 한글이포함되지않은파일경로: 실행된다. 
    - hwp5txt-script.py 한글이포함된파일경로: 실행된다. 한글 부분이 잘 들어간다. 
    - hwp5txt-script.py 한글이포함되지않은파일경로: 실행된다. 
- Script에서 os.system() 또는 subprocess.run()을 통해 실행한 경우:
    - hwp5txt 한글이포함된파일경로: 실행되지 않는다. 한글 부분이 ???로 들어가며 오류난다. 
    - hwp5txt 한글이포함되지않는파일경로: 실행된다. 
    - hwp5txt-script.py 한글이포함된파일경로: 실행되지 않는다. 파일경로에 한글이 포함된 것과 관계 없이 windows popup 오류가 뜨며 shell command 자체가 동작하지 않는다. 
    - hwp5txt-script.py 한글이포함되지않은파일경로: 마찬가지로 동작하지 않는다. 
    
실행환경:
- Windows 10 Pro 64bit
- VSCode, Git Bash for Windows terminal
- Python 3.7.6 64bit, on Poetry virtualenv. 

In [203]:
# exefile = 'hwp5txt-script.py' # Works on terminal, not on Jupyter. 
exefile = 'hwp5txt'

In [204]:
files_list = []

for _, _, files in os.walk(hwp_path):
    for file in files:
        files_list.append(file)

In [205]:
only_files = [f for f in os.listdir(hwp_path) if os.path.isfile(os.path.join(hwp_path, f))]

In [134]:
for file in only_files:
    old_file = os.path.join(hwp_path, file)
    new_file = os.path.join(hwp_tmp_path, ("bok_mpb_"+file[:10]+".hwp"))
    shutil.copy(old_file, new_file)

In [206]:
tmp_only_files = [f for f in os.listdir(hwp_tmp_path) if os.path.isfile(os.path.join(hwp_tmp_path, f))]

<font style='color:red;'>매우 중요하다. " " 안에 인자가 들어가야한다. ' '에 들어가면 실행되지 않는다.</font> 

매우 중요하다. " " 안에 인자가 들어가야한다. ' '에 들어가면 실행되지 않는다.

In [194]:
def quotation_result(filepath, filename):
    result = filepath + "\\" + filename
    return '"{}"'.format(result)

In [207]:
for file in tmp_only_files:
    command = [
        exefile, 
        quotation_result(hwp_tmp_path, file), 
        '>', 
        quotation_result(hwp2txt_path, file[:-4]+".txt"),
          ]
    command = " ".join(command)
    
    subprocess.run(command, shell=True)

# pdf to txt 변환 

Code Attribution: 김현진T, Fininsight

Requirement: tika=1.23.0 as of 2020.02.25 (os.setid doesn't work on Windows)

## 1. BOK - MPB minutes 

In [2]:
source_folder="./data_files/BOK_minutes/pdf/"

In [3]:
pdf_files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]

In [4]:
def pdf2txt(source_folder="./data_files/BOK_minutes/pdf/", output_folder="./data_files/BOK_minutes/pdf2txt/"):
    
    # 지정 폴더 내 파일 목록 조회 (파일만)
    pdf_files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
    #print(pdf_files)

    for pdf in pdf_files:
        try:
            pdf_filepath = source_folder + pdf
            pdf_tmp_filepath = output_folder + 'tmp.pdf'

            # pdf 파일을 text로 변환
            shutil.copyfile(pdf_filepath, pdf_tmp_filepath)
            parsedPDF = parser.from_file(pdf_tmp_filepath)["content"]
            #print(parsedPDF)

            #enter 삭제 
    #             parsedPDF = re.sub('\n', '', parser.from_file(pdf_tmp_filepath)["content"])

            output_filepath = (output_folder + pdf).replace('.pdf', '.txt')
            #print(output_filepath)
            with open(output_filepath, 'w',-1,'utf-8') as f:
                print(output_filepath)
                f.write(parsedPDF)
                f.close()
        except:
            print('Error Occurred at {}:'.format(pdf))
            print("Error:", sys.exc_info()[0])
            pass

In [5]:
pdf2txt()

./data_files/BOK_minutes/pdf2txt/2005-06-09_-_금융통화위원회 의사록(2005년도 제12차).txt
./data_files/BOK_minutes/pdf2txt/2005-06-23_-_금융통화위원회 의사록(2005년도 제13차).txt
./data_files/BOK_minutes/pdf2txt/2005-07-07_-_금융통화위원회 의사록(2005년도 제14차).txt
./data_files/BOK_minutes/pdf2txt/2005-08-11_-_금융통화위원회 의사록(2005년도 제17차).txt
./data_files/BOK_minutes/pdf2txt/2005-09-08_-_금융통화위원회 의사록(2005년도 제19차).txt
./data_files/BOK_minutes/pdf2txt/2005-09-22_-_금융통화위원회 의사록(2005년도 제20차).txt
./data_files/BOK_minutes/pdf2txt/2005-10-11_-_금융통화위원회 의사록(2005년도 제21차).txt
./data_files/BOK_minutes/pdf2txt/2005-11-10_-_금융통화위원회 의사록(2005년도 제24차).txt
./data_files/BOK_minutes/pdf2txt/2005-12-08_-_금융통화위원회 의사록(2005년도 제26차).txt
./data_files/BOK_minutes/pdf2txt/2005-12-22_-_금융통화위원회 의사록(2005년도 제27차).txt
./data_files/BOK_minutes/pdf2txt/2006-01-05_-_금융통화위원회 의사록(2006년도 제1차).txt
./data_files/BOK_minutes/pdf2txt/2006-01-12_-_금융통화위원회 의사록(2006년도 제2차).txt
./data_files/BOK_minutes/pdf2txt/2006-01-26_-_금융통화위원회 의사록(2006년도 제3차).txt
./data_files/BOK_minutes/pdf

## 2. Naver finance bond reports

매우 양이 많다. 주의할 것. 

In [9]:
source_folder="./data_files/NAVER_bond_reports/"

In [10]:
pdf_files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]

In [11]:
def pdf2txt(source_folder=source_folder, output_folder="./data_files/NAVER_bond_reports/pdf2txt/"):
    
    # 지정 폴더 내 파일 목록 조회 (파일만)
    pdf_files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
    #print(pdf_files)

    for pdf in pdf_files:
        try:
            pdf_filepath = source_folder + pdf
            pdf_tmp_filepath = output_folder + 'tmp.pdf'

            # pdf 파일을 text로 변환
            shutil.copyfile(pdf_filepath, pdf_tmp_filepath)
            parsedPDF = parser.from_file(pdf_tmp_filepath)["content"]
            #print(parsedPDF)

            #enter 삭제 
    #             parsedPDF = re.sub('\n', '', parser.from_file(pdf_tmp_filepath)["content"])

            output_filepath = (output_folder + pdf).replace('.pdf', '.txt')
            #print(output_filepath)
            with open(output_filepath, 'w',-1,'utf-8') as f:
                print(output_filepath)
                f.write(parsedPDF)
                f.close()
        except:
            print('Error Occurred at {}:'.format(pdf))
            print("Error:", sys.exc_info()[0])
            pass

In [12]:
pdf2txt()

./data_files/NAVER_bond_reports/pdf2txt/2008-04-01_대우증권대우증권 월간 채권투자 (4월).txt
./data_files/NAVER_bond_reports/pdf2txt/2008-04-07_대우증권돌아온 박스권.txt
./data_files/NAVER_bond_reports/pdf2txt/2008-04-07_대우증권시간차 공격.txt
./data_files/NAVER_bond_reports/pdf2txt/2008-04-14_대우증권총재의 변신은 무죄다!.txt
./data_files/NAVER_bond_reports/pdf2txt/2008-04-21_대우증권항상 열려는 있는 가능성.txt
./data_files/NAVER_bond_reports/pdf2txt/2008-04-28_대우증권FOMC 이후 원자재 가격 상승세 둔화 여부에 관심.txt
./data_files/NAVER_bond_reports/pdf2txt/2008-05-04_대우증권정부의 금리인하 요구에 한은의 부응 제스쳐 나타나며...txt
./data_files/NAVER_bond_reports/pdf2txt/2008-05-13_대우증권레인지의 상단부에서 기회를 다시 만들어가는 입장.txt
./data_files/NAVER_bond_reports/pdf2txt/2008-05-19_대우증권대우증권 5월 19일 채권 시황.txt
./data_files/NAVER_bond_reports/pdf2txt/2008-05-20_대우증권대우증권 5월 20일 채권 시황.txt
./data_files/NAVER_bond_reports/pdf2txt/2008-05-21_대우증권대우증권 5월 21일 채권 시황.txt
./data_files/NAVER_bond_reports/pdf2txt/2008-05-22_대우증권대우증권 5월 22일 채권 현황.txt
./data_files/NAVER_bond_reports/pdf2txt/2008-05-23_대우증권대우증권 5월 23일 채권 시황.